In [ ]:
import os
import sys
import math
import argparse
import numpy as np
from collections import Counter
from scipy import ndimage
from biosppy.signals.ecg import ecg

# Keras imports
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import tensorflow as tf

# ML4CVD Imports
from ml4cvd.arguments import parse_args
from ml4cvd.models import make_multimodal_multitask_model, train_model_from_generators
from ml4cvd.tensor_generators import TensorGenerator, big_batch_from_minibatch_generator, test_train_valid_tensor_generators
from ml4cvd.recipes import test_multimodal_scalar_tasks

# IPython imports
from IPython.display import Image
%matplotlib inline
import matplotlib.pyplot as plt

from scipy.stats import linregress

In [ ]:
tf.compat.v1.disable_eager_execution()

In [ ]:
def gradients_from_output(args, model, output_layer, output_index):
    K.set_learning_phase(1)
    input_tensor = model.input
    x = model.get_layer(output_layer).output[:, output_index]
    grads = K.gradients(x, input_tensor)[0]
    grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-6) # normalization trick: we normalize the gradient
    iterate = K.function(input_tensor, [x, grads])
    return iterate

def saliency_map(input_tensor, model, output_layer, output_index):
    get_gradients = gradients_from_output(args, model, output_layer, output_index)
    activation, grads = get_gradients(input_tensor)
    print('gradient shape:', grads.shape)
    return grads


# Load the training data and model
## Requirements
* Checkout branch `nd_jen_inference_tmaps`
* Download https://console.cloud.google.com/storage/browser/_details/ml4cvd/ndiamant/hrr_pretest_model.h5?project=broad-ml4cvd
* Download https://console.cloud.google.com/storage/browser/_details/ml4cvd/ndiamant/ensemble_hrr.csv?project=broad-ml4cvd and edit `TensorMap ecg-bike-hrr-ramp` in `tensor_from_file` so it points to the path of ensemble_hrr.csv
* Attach disk `ecg_bike_tensors`

In [ ]:
model_path = '/home/ndiamant/train_runs/hrr_swish_few_covs_ramp_warp_patient_bmi/hrr_swish_few_covs_ramp_warp_patient_bmi.h5'

In [ ]:
sys.argv = ['train', 
            '--tensors', '/mnt/disks/ecg-bike-tensors/2019-10-10/', 
            '--input_tensors', 'ecg_bike_shifted_8xdownsampled_10s','genetic_sex','ecg-bike-age','bmi',
            '--output_tensors', 'ecg-bike-hrr-ramp',
            '--batch_size', '32',
            '--epochs', '2',  
            '--learning_rate', '0.001',
            '--training_steps', '128',
            '--validation_steps', '10',
            '--test_steps', '1',
            '--model_file', model_path,
            '--id', 'hrr_saliency',
            '--test_csv', '/home/ndiamant/ecg_test_ids',
]
args = parse_args()
model = make_multimodal_multitask_model(**args.__dict__)
generate_train, generate_valid, generate_test = test_train_valid_tensor_generators(**args.__dict__)
test_data, test_labels, test_paths = big_batch_from_minibatch_generator(generate_test, args.test_steps)
;

# Get gradients and plot saliency maps of unmodified input

In [ ]:
test_tensor = list(test_data.values())
grads = saliency_map(test_tensor, model, 'output_hrr_continuous', 0)

In [ ]:
for i in range(1):
    fig, ax = plt.subplots(figsize=(12, 4))
    ctf = test_tensor[0][i]   
    g = grads[i].T.copy()
    g = ndimage.gaussian_filter(g, sigma=3)
    a = ax.imshow(g, cmap='plasma', aspect='auto', extent=[0, ctf.shape[0], ctf.min(), ctf.max()])
    plt.colorbar(a)
    ax.plot(ctf, 'k', lw=3)
    ax.set_title('Saliency map of HRR')
    plt.show()

# Methods to modify and align RR intervals

In [ ]:
def stretch_to_n_bpm(x, return_peaks=False, n=60):
    """
    stretches input ECG to n bpm
    """
    out = ecg(x.copy(), sampling_rate=500 // 4, show=False)
    hr = out[-1].mean() / 2
    t = np.arange(len(x))
    tp = np.arange(len(x)) * n / hr
    if return_peaks:
        return np.interp(tp, t, x), out[2] * hr / n
    return np.interp(tp, t, x)


def superimpose_grads(x, g, peaks):
    """
    Return each RR-interval of input ecg x.
    xs is each stretched RR interval of ECG
    sum_g is stretched gradients averaged over each RR-interval
    sum_g is kinda like SELECT SUM(gradients) GROUP BY RR-interval
    """
    xs = []
    n = 100
    sum_g = np.zeros(n)
    for i in range(1, len(peaks) - 1):
        j, k = int(peaks[i - 1]), int(peaks[i])
        if k > len(x):
            continue
        stretch = lambda z: np.interp(np.linspace(0, 1, n), np.linspace(0, 1, k - j), z[j: k]) 
        xs.append(stretch(x))
        sum_g += stretch(g)
    return xs, sum_g

In [ ]:
# Test of stretching ECG to 40 bpm
a = test_tensor[0][1][:,0].copy()
plt.plot(a, label='og')
plt.plot(stretch_to_n_bpm(a, n=40), label='stretched')
plt.legend()
plt.show()

# Stretch input data and calculate gradients

In [ ]:
test_data_stretched = test_data.copy()
peaks = []
new_bpm = 50
for i in range(len(test_data_stretched['input_full_continuous'])):
    test_data_stretched['input_full_continuous'][i, :, 0], p = stretch_to_n_bpm(test_data['input_full_continuous'][i, :, 0].copy(), return_peaks=True, n=new_bpm)
    peaks.append(p)

In [ ]:
test_tensor = list(test_data_stretched.values())
grads = saliency_map(test_tensor, model, 'output_hrr_continuous', 0)

# Plot saliency map of stretched inputs

In [ ]:
for i in range(1):
    fig, ax = plt.subplots(figsize=(12, 4))
    ctf = test_tensor[0][i]  
    g = grads[i].T.copy()
    g = ndimage.gaussian_filter(g, sigma=3)
    a = ax.imshow(g, cmap='plasma', aspect='auto', extent=[0, ctf.shape[0], ctf.min(), ctf.max()])
    plt.colorbar(a)
    ax.plot(ctf, 'k', lw=3)
    ax.set_title('Saliency map of HRR')
    plt.show()

# Plot saliency maps aligned to RR-interval

In [ ]:
for i in range(16):
    fig, ax = plt.subplots(figsize=(12, 4))
    ctf = test_tensor[0][i]
    hrr_pred = model.predict([x[i: i + 1] for x in test_tensor])[0, 0]
    hrr_pred = hrr_pred * 15 + 25
    g = grads[i].T.copy()
    xs, sum_g = superimpose_grads(ctf[:, 0], g.T[:, 0], peaks[i]) 
    g = ndimage.gaussian_filter(sum_g[np.newaxis] / len(xs), sigma=2)
    a = ax.imshow(g, cmap='plasma', aspect='auto', extent=[0, xs[0].shape[0], np.min(xs), np.max(xs)])
    plt.colorbar(a)
    for x in xs:
        ax.plot(x, 'k', lw=3, alpha=.5)
    ax.set_title(f'Saliency map of HRR, predicted is {hrr_pred:.2f}')
    plt.show()

# Modify ECG by gradient to increase predicted HRR

In [ ]:
def ma(x, w):
    """moving average"""
    s = x.shape
    return (np.convolve(x.flatten(), np.ones(w), 'same') / w).reshape(s)

In [ ]:
# modify input by grad and see change in predicted hrr
idx = 0
ctf = test_tensor[0][idx].copy()
inp = [x[idx: idx + 1].copy() for x in test_tensor]
g = grads[idx].T.copy()
delta = 5e-2
w = 10
fig, ax = plt.subplots(figsize=(16, 8))
n = 10
for i in range(n): 
    hrr_pred = model.predict(inp)[0, 0]
    hrr_pred = hrr_pred * 15 + 25        
    inp[0] += ma(g.T, w) * delta
    ctf += ma(g.T, w) * delta
    xs, sum_g = superimpose_grads(ctf[:, 0], g.T[:, 0], peaks[idx]) 
    g2 = ndimage.gaussian_filter(sum_g[np.newaxis] / len(xs), sigma=2)
    a = ax.imshow(g2, cmap='plasma', aspect='auto', extent=[0, xs[0].shape[0], np.min(xs), np.max(xs)])
    c = i / n, i / n, i / n
    ax.plot(np.median(xs, axis=0), c=c, alpha=.9, label=f'hrr is {hrr_pred:.2f}')
plt.colorbar(a)
plt.legend()
plt.show()